# Using Dask with Jupyter Widgets for Visualizing Queries on Large Datasets
    Scott Little, Valassis Digital
    Jupyter Day in the Triangle, 2018

### Key takeaways

1) Splitting files for multithreading / remote storage of "big data"
<br>
2) Speed up due to parallel processing
<br>
3) Query data with Jupyter Widgets for data exploration

### Definitions

**S3**: Remote (cloud) storage by Amazon Web Services (AWS) that uses key-based (dictionary) multi-threaded access
<br>
**Dask**: Parallel processing library for Python that enables out-of-core calculations
<br>
**Out-of-core**: Computation on data not fully loaded into RAM
<br>
**Jupyter Widget**: Interact with output (plots, etc.) in an HTML way
<br>
**Parquet**: Column based file format. Can be compressed and split. Used in commonly in the Spark universe.
<br>
**Conda Environment**: Loads necessary Python packages. Highly recommended.
<br>
**Big data**: Larger than what will fit in your laptop's RAM
<br>
**Bigger data**: Larger than what will fit in your laptop's hard drive

### First step (not shown): download datasets and upload to AWS S3

Kiva loan data from http://build.kiva.org/docs/data/snapshots
<br>
**loans**: 2.41 GB (1,419,607 rows)
<br>
**lenders**: 269 MB (2,349,174 rows)
<br>
**loans-lenders**: 357 MB (28,293,931 rows)
<br>
**Cost of join** ~ 28,293,931 / 1,419,607 * 2.41 GB ~ **48 GB**

### Imports

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import sleep
import dask.dataframe as dd
%matplotlib inline

### Read in from AWS S3

In [5]:
%time loans = dd.read_parquet( 's3://bigbucker/kiva_loans.parquet' )

CPU times: user 1.32 s, sys: 190 ms, total: 1.51 s
Wall time: 35 s


In [75]:
%time loans.head()

CPU times: user 161 ms, sys: 86.9 ms, total: 248 ms
Wall time: 729 ms


,LOAN_ID,LOAN_NAME,ORIGINAL_LANGUAGE,DESCRIPTION,DESCRIPTION_TRANSLATED,FUNDED_AMOUNT,LOAN_AMOUNT,STATUS,IMAGE_ID,VIDEO_ID,...,LENDER_TERM,NUM_LENDERS_TOTAL,NUM_JOURNAL_ENTRIES,NUM_BULK_ENTRIES,TAGS,BORROWER_NAMES,BORROWER_GENDERS,BORROWER_PICTURED,REPAYMENT_INTERVAL,DISTRIBUTION_MODEL
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",None,125.0,125.0,funded,1511246.0,NaN,...,7.0,3,2,1,None,Aivy,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,1507947.0,NaN,...,8.0,11,2,1,None,Idalia Marizza,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,None,400.0,400.0,funded,1512245.0,NaN,...,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",Aasia,female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,1514419.0,NaN,...,14.0,21,2,1,user_favorite,Gulmira,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,None,425.0,425.0,funded,1506830.0,NaN,...,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",Ricky\t,male,true,bullet,field_partner


In [91]:
%time loans.shape[0].compute()

CPU times: user 24.7 s, sys: 7.57 s, total: 32.2 s
Wall time: 1min 15s


1419607

In [59]:
%time loans_lid = loans.set_index('LOAN_ID')

CPU times: user 26.6 s, sys: 6.31 s, total: 32.9 s
Wall time: 1min 1s


In [60]:
%time loans_lid.head()

CPU times: user 41.3 s, sys: 10.9 s, total: 52.2 s
Wall time: 1min 27s


,LOAN_NAME,ORIGINAL_LANGUAGE,DESCRIPTION,DESCRIPTION_TRANSLATED,FUNDED_AMOUNT,LOAN_AMOUNT,STATUS,IMAGE_ID,VIDEO_ID,ACTIVITY_NAME,...,LENDER_TERM,NUM_LENDERS_TOTAL,NUM_JOURNAL_ENTRIES,NUM_BULK_ENTRIES,TAGS,BORROWER_NAMES,BORROWER_GENDERS,BORROWER_PICTURED,REPAYMENT_INTERVAL,DISTRIBUTION_MODEL
LOAN_ID,,,,,,,,,,,,,,,,,,,,,
84,Justine,English,<i>The following description was written by Mo...,None,500.0,500.0,funded,241.0,NaN,Butcher Shop,...,6.0,5,34,17,user_favorite,Justine,male,true,bullet,field_partner
85,Geoffrey,English,<i>The following description was written by Mo...,None,500.0,500.0,funded,252.0,NaN,Food Production/Sales,...,6.0,3,32,16,None,Geoffrey,male,true,bullet,field_partner
86,Apollo,English,<i>The following description was written by Mo...,None,500.0,500.0,funded,243.0,NaN,Animal Sales,...,6.0,3,30,15,None,Apollo,male,true,bullet,field_partner
88,Eunice,English,Eunice received 100 dollar grant from Village ...,None,300.0,300.0,funded,253.0,NaN,Clothing Sales,...,6.0,4,24,12,None,Eunice,female,true,bullet,field_partner
89,Rose,English,<i>\r\nThe following description was written b...,None,500.0,500.0,funded,246.0,NaN,Food Production/Sales,...,6.0,6,28,14,None,Rose,female,true,bullet,field_partner


### Lenders

In [7]:
%time lenders = dd.read_parquet( 's3://bigbucker/kiva_lenders.parquet' )

CPU times: user 1.83 s, sys: 174 ms, total: 2.01 s
Wall time: 34.1 s


In [54]:
lenders.head()

,PERMANENT_NAME,DISPLAY_NAME,MAIN_PIC_ID,CITY,STATE,COUNTRY_CODE,MEMBER_SINCE,PERSONAL_URL,OCCUPATION,LOAN_BECAUSE,OTHER_INFO,LOAN_PURCHASE_NUM,INVITED_BY,NUM_INVITED
0,qian3013,Qian,NaN,None,None,None,1461300457,None,None,None,None,1.0,None,0
1,reena6733,Reena,NaN,None,None,None,1461300634,None,None,None,None,9.0,None,0
2,mai5982,Mai,NaN,None,None,None,1461300853,None,None,None,None,NaN,None,0
3,andrew86079135,Andrew,NaN,None,None,None,1461301091,None,None,None,None,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,None,None,None,1461301154,None,None,None,None,NaN,None,0


In [92]:
%time lenders.shape[0].compute()

CPU times: user 7.99 s, sys: 1.16 s, total: 9.16 s
Wall time: 26.3 s


2349174

### Loans by Lenders

In [8]:
%time loans_lenders = dd.read_parquet( 's3://bigbucker/kiva_loans_lenders.parquet' )

CPU times: user 1.06 s, sys: 120 ms, total: 1.18 s
Wall time: 23.6 s


In [36]:
loans_lenders.head()

,LOAN_ID,LENDERS
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


#### setting indexes
makes things faster / easier

In [37]:
ll = loans_lenders.set_index('LOAN_ID')

In [38]:
ll.head()

,LENDERS
LOAN_ID,
84,"ward, michael, brooke"
85,"michael, patrick"
86,"michael, METS, erik"
88,"michael, susan, ryan"
89,"hope, michael, dan, ryan"


### 'Explode' (like in Spark)

In [82]:
def func(df):
    return df['LENDERS'].map(lambda x:x.split(',')).apply(pd.Series, 1).stack().reset_index(level=1, drop=True).rename('LENDERS')

result = ll.map_partitions(func)

In [83]:
%time result.head()

CPU times: user 11.4 s, sys: 1.53 s, total: 12.9 s
Wall time: 13.3 s


LOAN_ID
84        ward
84     michael
84      brooke
85     michael
85     patrick
Name: LENDERS, dtype: object

In [58]:
%time result.shape[0].compute()

CPU times: user 10min 53s, sys: 1min 4s, total: 11min 58s
Wall time: 10min 13s


28293931

### Store result

In [85]:
%time result.reset_index().to_parquet('s3://bigbucker/kiva_loans_lenders_exploded.parquet', compression='gzip')

CPU times: user 11min 21s, sys: 1min 8s, total: 12min 29s
Wall time: 10min 11s


**Note**: Power of Dask
<br>
Similar "in-core" code:
```
df_final = pd.DataFrame()
for i in range(df.shape[0]):
    df_temp = pd.DataFrame( df.iloc[i].LENDERS.split(',') )
    df_temp.columns = ['LENDERS']
    df_temp['LOAN_ID'] = df.iloc[i].LOAN_ID
    df_final = pd.concat( [df_final, df_temp], ignore_index=True )
```
I let this run ~**45 min** before killing it.

In [86]:
%time loans_lenders_exploded = dd.read_parquet( 's3://bigbucker/kiva_loans_lenders_exploded.parquet' )

CPU times: user 1.12 s, sys: 143 ms, total: 1.26 s
Wall time: 17.6 s


In [90]:
%time loans_lenders_exploded.shape[0].compute()

CPU times: user 8.82 s, sys: 1.99 s, total: 10.8 s
Wall time: 15.8 s


28293931

In [93]:
%time loans_lenders_exploded_lid = loans_lenders_exploded.set_index('LOAN_ID')

CPU times: user 22.1 s, sys: 12 s, total: 34.1 s
Wall time: 16.1 s


In [112]:
loans[['LOAN_ID','LOAN_AMOUNT','STATUS']].head()

,LOAN_ID,LOAN_AMOUNT,STATUS
0,657307,125.0,funded
1,657259,400.0,funded
2,658010,400.0,funded
3,659347,625.0,funded
4,656933,425.0,funded


### Optional join

In [118]:
%time loans_join = loans[['LOAN_ID','STATUS']].merge(loans_lenders_exploded, on='LOAN_ID', how='inner')

CPU times: user 24.3 ms, sys: 2.09 ms, total: 26.4 ms
Wall time: 24.9 ms


In [129]:
%time loans_join.head()

CPU times: user 1min 20s, sys: 19.4 s, total: 1min 39s
Wall time: 1min 45s


,LOAN_ID,STATUS,LENDERS
0,573218,funded,kenneth2234
1,573218,funded,andrew7452
2,573218,funded,ronald1747
3,573218,funded,jason6815
4,573218,funded,krista6045


In [132]:
%time loans_join.to_parquet('s3://bigbucker/kiva_loans_join.parquet')

CPU times: user 2min 9s, sys: 27.9 s, total: 2min 37s
Wall time: 2min 29s


In [133]:
%time loans_join = dd.read_parquet( 's3://bigbucker/kiva_loans_join.parquet' )

CPU times: user 1.06 s, sys: 132 ms, total: 1.19 s
Wall time: 10.6 s


In [134]:
%time loans_join.head()

CPU times: user 94.5 ms, sys: 28.6 ms, total: 123 ms
Wall time: 456 ms


,LOAN_ID,STATUS,LENDERS
0,998569,funded,nicolas3628
1,998569,funded,janet6829
2,998569,funded,shaunna2345
3,998569,funded,jonathan2303
4,998569,funded,dewittkavanagh


### Jupyter Widgets

In [66]:
# Some css formatting for the displayed dataframe
from IPython.display import HTML, display

def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])

In [67]:
# Setting some necessary ipywidgets variables
from ipywidgets import widgets
import warnings #suppress warnings in app
warnings.filterwarnings('ignore')

inputText = widgets.Text()
inputText.placeholder = 'Enter species here'

outputText = widgets.HTML()

In [135]:
def searchWidget(sender):
    
    outputText.value = 'Calculating...'
    
    # do something (more) useful here
    loans_filtered = loans_join[loans_join['STATUS']==inputText.value]
    
    # dataframe CSS properties
    styles = [
        hover(),
        dict(selector="th", props=[("font-size", "125%"),
                                   ("text-align", "center"),
                                  ("transform", "translate(0%,-20%) rotate(-5deg)")
                                  ]),
        dict(selector=".row_heading, .blank", props= [('display', 'none;')])
    ]
    
    # replace outputText.value with HTML string
    outputText.value = (loans_filtered#.sort_values('sepal_length',ascending=False)
                        .head(10)
                        .style.set_table_styles(styles).render()
                       )

In [136]:
# what to do when enter is pressed (try "on_trait_change")
inputText.on_submit(searchWidget)

In [138]:
# display actual output of app
display(HTML('<b>Enter loan status:</b><br>'))
display(inputText)
display(HTML('<b>Top 10 matches:</b><br>'))
display(outputText)

Text(value='funded', placeholder='Enter species here')

HTML(value='<style  type="text/css" >\n    #T_6ba7858c_e76d_11e8_bdb3_8c8590c95d5e tr:hover {\n          backg…

**Note**: For the Jupyter notebook, we could have incorporated a slider and a plot as well.